In [1]:
!pip install shutup==0.2.0
!pip install spacy==3.5.2
!pip install gensim==4.3.1
!pip install scipy==1.10.1
!pip install pyemd==1.0.0
!pip install scikit-learn==1.2.2
!pip install nltk==3.8.1
!pip install datasets==2.12.0
!pip install jupyter==1.0.0
!pip install POT==0.9.0
!pip install keras==2.12.0
!pip install torch==2.0.0
!pip install accelerate==0.18.0
!pip install transformers==4.28.1

!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.6/662.6 kB 10.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd "drive/MyDrive/Colab Notebooks/nlp proj"

/content/drive/MyDrive/Colab Notebooks/nlp proj


In [58]:
import shutup; shutup.please()
import spacy
from spacy.lang.en import English
import numpy as np
import gensim
from gensim.models import KeyedVectors
import gensim.downloader as api
from scipy.spatial.distance import cosine, hamming, euclidean, cityblock
from scipy.stats import spearmanr, wasserstein_distance
from pyemd import emd
from sklearn.metrics.pairwise import cosine_similarity, paired_distances, euclidean_distances
import nltk
from nltk.corpus import stopwords
import string
from datasets import load_dataset
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.functional import norm
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
import time
from tqdm import tqdm
from siamese_sts.data_loader import STSData
from siamese_sts.siamese_net import SiameseBiLSTMAttention
from siamese_sts.trainer.train import train_model
from siamese_sts.data_loader.preprocess import Preprocess

In [42]:
# Load spacy model
nlp = spacy.load('en_core_web_sm')
disabled = [component[0] for component in nlp.components if component[0] != 'tok2vec']

In [43]:
# Load word2vec model
w2v_model = api.load('word2vec-google-news-300')

columns_mapping = {
    "sent1": "sentence_A",
    "sent2": "sentence_B",
    "label": "relatedness_score",
}

In [44]:
dataset_name = "sick"
dataset = load_dataset(dataset_name)
train_dataset = dataset['train'].to_pandas()
test_dataset = dataset['test'].to_pandas()
cur_dataset = train_dataset

  0%|          | 0/3 [00:00<?, ?it/s]

In [59]:
preprocess_ = Preprocess('siamese_sts/data_loader/stopwords-en.txt')

def preprocess_text(text):
    text = preprocess_.remove_usernames(text)
    text = preprocess_.remove_punctuations(text)
    text = preprocess_.remove_stopwords(text)
    return text

In [60]:
# Compute distance between texts using WMD
def compute_wmd_distance(text1, text2):
    tokenized_text1 = text1.split()
    tokenized_text2 = text2.split()
    try:
        distance = w2v_model.wmdistance(tokenized_text1, tokenized_text2)
        return distance
    except ValueError:
        return 0

In [61]:
def tokenize_text(text):
    vector = nlp(text, disable=disabled).vector
    return vector

In [62]:
# Compute distance between texts using cosine distance
def compute_cosine_distance(vector1, vector2):
    try:
        distance = cosine(vector1, vector2)
        return distance
    except ValueError:
            return 0

In [63]:
# Compute distance between texts using Hamming distance
def compute_hamming_distance(vector1, vector2):
    try:
        distance = hamming(vector1, vector2)
        return distance
    except ValueError:
        return 0

In [64]:
# Compute distance between texts using Euclidean distance
def compute_euclidean_distance(vector1, vector2):
    try:
        distance = euclidean(vector1, vector2)
        return distance
    except ValueError:
        return 0

In [65]:
# Compute distance between texts using Manhattan distance
def compute_manhattan_distance(vector1, vector2):
    try:
        distance = cityblock(vector1, vector2)
        return distance
    except ValueError:
        return 0

In [66]:
# Compute distance between texts using Earth Mover's Distance
def compute_emd_distance(vector1, vector2):
    try:
        distance = wasserstein_distance(vector1, vector2)
        return distance
    except ValueError:
        return 0

In [67]:
preprocessed_texts1 = cur_dataset[columns_mapping['sent1']].apply(preprocess_text)
preprocessed_texts2 = cur_dataset[columns_mapping['sent2']].apply(preprocess_text)

In [68]:
tokenized_texts1 = preprocessed_texts1.apply(tokenize_text)
tokenized_texts2 = preprocessed_texts2.apply(tokenize_text)

In [72]:
start_time = time.time()
wmd_distances = preprocessed_texts1.combine(preprocessed_texts2, compute_wmd_distance)
wmd_time = time.time() - start_time
start_time = time.time()
cosine_distances = tokenized_texts1.combine(tokenized_texts2, compute_cosine_distance)
cosine_time = time.time() - start_time
start_time = time.time()
hamming_distances = tokenized_texts1.combine(tokenized_texts2, compute_hamming_distance)
hamming_time = time.time() - start_time
start_time = time.time()
euclidean_distances_ = tokenized_texts1.combine(tokenized_texts2, compute_euclidean_distance)
euclidean_time = time.time() - start_time
start_time = time.time()
manhattan_distances = tokenized_texts1.combine(tokenized_texts2, compute_manhattan_distance)
manhattan_time = time.time() - start_time
start_time = time.time()
emd_distances = tokenized_texts1.combine(tokenized_texts2, compute_emd_distance)
emd_time = time.time() - start_time

In [73]:
wmd_correlation, _ = spearmanr(wmd_distances, cur_dataset[columns_mapping['label']])
cosine_correlation, _ = spearmanr(cosine_distances, cur_dataset[columns_mapping['label']])
hamming_correlation, _ = spearmanr(hamming_distances, cur_dataset[columns_mapping['label']])
euclidean_correlation, _ = spearmanr(euclidean_distances_, cur_dataset[columns_mapping['label']])
manhattan_correlation, _ = spearmanr(manhattan_distances, cur_dataset[columns_mapping['label']])
emd_correlation, _ = spearmanr(emd_distances, cur_dataset[columns_mapping['label']])

In [74]:
print("WMD correlation:", wmd_correlation, "Time:", wmd_time)
print("Cosine similarity correlation:", cosine_correlation, "Time:", cosine_time)
print("Hamming correlation:", hamming_correlation, "Time:", hamming_time)
print("Euclidean correlation:", euclidean_correlation, "Time:", euclidean_time)
print("Manhattan correlation:", manhattan_correlation, "Time:", manhattan_time)
print("EMD correlation:", emd_correlation, "Time:", emd_time)

wmd_correlation = abs(wmd_correlation)
cosine_correlation = abs(cosine_correlation)
hamming_correlation = abs(hamming_correlation)
euclidean_correlation = abs(euclidean_correlation)
manhattan_correlation = abs(manhattan_correlation)
emd_correlation = abs(emd_correlation)

# Determine the best approach based on the highest correlation coefficient
max_correlation = max(wmd_correlation, cosine_correlation, hamming_correlation, euclidean_correlation, manhattan_correlation, emd_correlation)
if max_correlation == wmd_correlation:
    print("WMD distance is the best approach.")
elif max_correlation == cosine_correlation:
    print("Cosine similarity distance is the best approach.")
elif max_correlation == hamming_correlation:
    print("Hamming distance is the best approach.")
elif max_correlation == euclidean_correlation:
    print("Euclidean distance is the best approach.")
elif max_correlation == manhattan_correlation:
    print("Manhattan distance is the best approach.")
else:
    print("EMD distance is the best approach.")

WMD correlation: -0.568819990570633 Time: 3.2531816959381104
Cosine similarity correlation: -0.41558549103168596 Time: 0.2979443073272705
Hamming correlation: -0.07800115713400245 Time: 0.11627888679504395
Euclidean correlation: -0.39692602055181825 Time: 0.0842134952545166
Manhattan correlation: -0.39800411064234925 Time: 0.07181262969970703
EMD correlation: -0.2667198600870091 Time: 0.38802027702331543
WMD distance is the best approach.


In [ ]:
columns_mapping_spec = {
    "sent1": "sentence_A",
    "sent2": "sentence_B",
    "label": "wmdistance",
}

def prevocab(dataset_):
  dataset_[columns_mapping_spec['label']] = dataset_['clean_sent1'].combine(dataset_['clean_sent2'], compute_wmd_distance)

  return dataset_

In [ ]:
sick_data = STSData(dataset_name=dataset_name, columns_mapping=columns_mapping, prevocab=prevocab, vector_cache='../../.vector_cache')
sick_dataloaders = sick_data.get_data_loader()
batch_size = 16
output_size = 1
hidden_size = 128
vocab_size = len(sick_data.vocab)
embedding_size = 300
embedding_weights = sick_data.vocab.vectors
lstm_layers = 2
learning_rate = 1e-1
fc_hidden_size = 64
max_epochs = 5
bidirectional = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## self attention config
self_attention_config = {
    "hidden_size": 150,  ## refers to variable 'da' in the ICLR paper
    "output_size": 20,  ## refers to variable 'r' in the ICLR paper
    "penalty": 0.6,  ## refers to penalty coefficient term in the ICLR paper
}

## init siamese lstm
siamese_lstm_attention = SiameseBiLSTMAttention(
    batch_size=batch_size,
    output_size=output_size,
    hidden_size=hidden_size,
    vocab_size=vocab_size,
    embedding_size=embedding_size,
    embedding_weights=embedding_weights,
    lstm_layers=lstm_layers,
    self_attention_config=self_attention_config,
    fc_hidden_size=fc_hidden_size,
    device=device,
    bidirectional=bidirectional,
)

## define optimizer and loss function
optimizer = torch.optim.Adam(params=siamese_lstm_attention.parameters())

In [ ]:
train_model(
        model=siamese_lstm_attention,
        optimizer=optimizer,
        dataloader=sick_dataloaders,
        max_epochs=max_epochs,
        config_dict={
            "device": device,
            "model_name": "siamese_lstm_attention",
            "self_attention_config": self_attention_config,
        },
    )